## ファイルから物性値を取得し、流量計算に適した形に換算するモジュール

### 1. 物性値ファイルをインポートするモジュール

### 物性値の名称とモジュール名を一致させるためのモジュール

#### 物性値モジュールリスト  
予め物性値のモジュールはincludeし、辞書型で"物性値名"とモジュールを合致させておく必要がある。

### 2. 取得した物性値を計算用に換算するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

### 2.1 材料実質部の空隙率・密度・比熱などの物性値を取得するクラス

#### 各物性値リストから値を取り出す関数

### 2.2 材料の水分特性（吸着力）などの情報を取得するクラス

#### 各物性値リストから値を取り出す関数

### 2.3 材料特性・含水状態からの熱・水分の移動係数を取得・換算するクラス

#### 熱物性値  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：
$$
    \dot q = -\lambda・grad\space T
$$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$\dot m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$\dot m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  

#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：
$$
    m_v = -\delta_p・grad\space p_v
$$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：
$$
    \dot m_v = -\delta_v・grad\space \rho_v
$$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：
$$
    \dot m_m = -k_T・grad\space T
$$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：
$$
    \dot m_m = -\rho_0 D_T・grad\space T
$$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：
$$
    \dot m_v = -\lambda^{'}_{\mu g}・grad\space \mu
$$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：
$$
    \dot m_v = -\lambda^{'}_{Tg}・grad\space \mu
$$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：
$$
    \dot m_m = -k_m・grad\space s
$$

$D_w$：水分拡散係数[m2/s]  
定義：
$$
    \dot m_m = -\rho_0 D_w・grad\space u
$$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：
$$
    \dot m_v = -\lambda^{'}_{\mu l}・grad\space \mu
$$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


#### 重力加速度


$\frac{\partial P_v}{\partial \mu}$ ：水蒸気圧を水分化学ポテンシャルで偏微分したもの  
$\frac{\partial P_v}{\partial T}$：水蒸気圧を温度で偏微分したもの  

In [1]:
module property_conversion

#####################################################
# 辞書型データの作成
# 材料物性データを保持している辞書型リストの作成
prop_list = Dict{String, Any}()

# 材料物性データのロード
include(string("./material_property/bentheimer_sandstone.jl"))
include(string("./material_property/benchmark_EN15026.jl"))
include(string("./material_property/tuff_motomachi.jl"))
include(string("./material_property/tuff_motomachi_v2.jl"))
include(string("./material_property/tuff_motomachi_v3.jl"))
include(string("./material_property/loamy_sand.jl"))
include(string("./material_property/plainfield_sand.jl"))
include(string("./material_property/sandy_clay_loam.jl"))
include(string("./material_property/mud_wall.jl"))
include(string("./material_property/plywood.jl"))
include(string("./material_property/extruded_polystyrene.jl"))
include(string("./material_property/asphalt_felt.jl"))
include(string("./material_property/ceramic_siding.jl"))
include(string("./material_property/breathable_water_proof_sheet.jl"))
include(string("./material_property/glass_wool_16K.jl"))
include(string("./material_property/glass_wool_16K_original.jl"))
include(string("./material_property/glass_wool_hokoi.jl"))
include(string("./material_property/moisture_proof_sheet.jl"))
include(string("./material_property/mortar.jl"))
include(string("./material_property/plasterboard.jl"))
include(string("./material_property/structural_plywood.jl"))
include(string("./material_property/vented_air_space.jl"))
include(string("./material_property/concrete_goran.jl"))
include(string("./material_property/metal_plate.jl"))
include(string("./material_property/paper.jl"))
include(string("./material_property/ALC.jl"))

# 辞書の登録
prop_list["bentheimer_sandstone"] = bentheimer_sandstone
prop_list["benchmark_EN15026"] = benchmark_EN15026
prop_list["tuff_motomachi"] = tuff_motomachi
prop_list["tuff_motomachi_v2"] = tuff_motomachi_v2
prop_list["tuff_motomachi_v3"] = tuff_motomachi_v3
prop_list["loamy_sand"] = loamy_sand
prop_list["plainfield_sand"] = plainfield_sand
prop_list["sandy_clay_loam"] = sandy_clay_loam
prop_list["mud_wall"] = mud_wall
prop_list["plywood"] = plywood
prop_list["extruded_polystyrene"] = extruded_polystyrene
prop_list["asphalt_felt"] = asphalt_felt
prop_list["ceramic_siding"] = ceramic_siding
prop_list["extruded_polystyrene"] = extruded_polystyrene
prop_list["breathable_water_proof_sheet"] = breathable_water_proof_sheet
prop_list["glass_wool_16K"] = glass_wool_16K
prop_list["glass_wool_16K_original"] = glass_wool_16K_original
prop_list["glass_wool_hokoi"] = glass_wool_hokoi
prop_list["moisture_proof_sheet"] = moisture_proof_sheet
prop_list["mortar"] = mortar
prop_list["plasterboard"] = plasterboard
prop_list["structural_plywood"] = structural_plywood
prop_list["vented_air_space"] = vented_air_space
prop_list["concrete_goran"] = concrete_goran
prop_list["metal_plate"] = metal_plate
prop_list["paper"] = paper
prop_list["ALC"] = ALC

#####################################################
const grav = 9.806650

include("./function/vapour.jl")
include("./material_property/liquid_water.jl")

# 関係式
# 水蒸気圧の水分化学ポテンシャル微分
cal_dpdmiu( cell ) = cal_Pvs( temp = cell.temp ) * exp( cell.miu / Rv / cell.temp ) / Rv / cell.temp

# 水蒸気圧の温度微分
cal_dpdT( cell ) = exp( cell.miu / Rv / cell.temp ) * ( cal_DPvs( temp = cell.temp ) - cal_Pvs( temp = cell.temp ) * cell.miu / Rv / ( cell.temp^2.0 ) )

# 熱容量（液水の熱容量を加えた場合）
cal_crow_add_water( cell ) = cell.crow + water_property.Cr * water_property.row * cell.phi


######################################################
# 材料特性値の取得
# 空隙率
get_psi(material_name::String) = prop_list[material_name].psi

# 比熱
get_C(material_name::String) = prop_list[material_name].C

# 材料密度
get_row(material_name::String) = prop_list[material_name].row

# 熱容量
get_crow(material_name::String) = prop_list[material_name].crow


#######################################################
# 水分特性値の取得
# 含水率
get_phi(cell, material_name::String) = prop_list[material_name].get_phi(cell)

# 含水率の水分化学ポテンシャル微分
get_dphi(cell, material_name::String) = prop_list[material_name].get_dphi(cell)

# 含水率から水分化学ポテンシャルを計算する関数
get_miu_by_phi(cell, material_name::String) = prop_list[material_name].get_miu_by_phi(cell)
get_miu_by_phi(phi::Float64, material_name::String) = prop_list[material_name].get_miu_by_phi(phi = phi)


#######################################################
# 移動係数の取得
### 熱 ###
# 熱伝導率（固体実質部）
get_lam(cell, material_name::String) = prop_list[material_name].get_lam(cell)

### 水（液） ###
# 液相水分伝導率
get_dw(cell, material_name::String) = prop_list[material_name].get_dw(cell)
get_dw_by_ldml(cell, material_name::String) = prop_list[material_name].get_ldml(cell) / water_property.row * grav

# 水分化学ポテンシャル勾配に対する液相水分伝導率
get_ldml(cell, material_name::String) = prop_list[material_name].get_ldml(cell)
get_ldml_by_dw(cell, material_name::String) = prop_list[material_name].get_dw(cell) * water_property.row / grav

### 水（気） ###
# 気相水分伝導率
get_dp(cell, material_name::String) = prop_list[material_name].get_dp(cell)
get_dp_by_ldmg(cell, material_name::String) = prop_list[material_name].get_ldmg(cell) / cal_dpdmiu( cell ) + prop_list[material_name].get_ldtg(cell) / cal_dpdT( cell )

# 水分化学ポテンシャル勾配に対する気相水分伝導率
get_ldmg(cell, material_name::String) = prop_list[material_name].get_ldmg(cell)
get_ldmg_by_dp(cell, material_name::String) = prop_list[material_name].get_dp(cell) * cal_dpdmiu( cell )

# 温度勾配に対する気相水分伝導率
get_ldtg(cell, material_name::String) = prop_list[material_name].get_ldtg(cell)
get_ldtg_by_dp(cell, material_name::String) = prop_list[material_name].get_dp(cell) * cal_dpdT( cell )


end

Main.property_conversion

## 便利な形へ変更

In [2]:

pc = property_conversion

include("./material_property/liquid_water.jl")

# 各物性を保持する構造体
mutable struct get_function_material_property
    psi ; C ; row ; crow ; 
    phi ; dphi ; miu ;
    lam ; dw ; dp ;
    ldml;ldmg; ldtg
    get_function_material_property() = new()
end

# 関数形で保持する場合はこちら
function set_function_material_property( material_name )
    prop = get_function_material_property()
    prop.psi  =  pc.get_psi
    prop.C    =  pc.get_C
    prop.row  =  pc.get_row
    prop.crow =  pc.cal_crow_add_water
    prop.phi  =  pc.get_phi
    prop.dphi =  pc.get_dphi
    prop.miu  =  pc.get_miu_by_phi
    prop.lam  =  pc.get_lam
    prop.dw   =  try; pc.get_dw;   catch; pc.get_dw_by_ldml; end
    prop.dp   =  try; pc.get_dp;   catch; pc.get_dp_by_ldmg; end
    prop.ldml =  try; pc.get_ldml; catch; pc.get_ldml_by_dw; end
    prop.ldmg =  try; pc.get_ldmg; catch; pc.get_ldmg_by_dp; end
    prop.ldtg =  try; pc.get_ldtg; catch; pc.get_ldtg_by_dp; end  
    return prop
end

# 物性値と移動係数を分離
#　熱水分物性
function set_function_hygrothermal_property( material_name )
    prop = get_function_material_property()
    prop.psi  =  pc.get_psi
    prop.C    =  pc.get_C
    prop.row  =  pc.get_row
    prop.crow =  pc.cal_crow_add_water
    prop.phi  =  pc.get_phi
    prop.dphi =  pc.get_dphi
    prop.miu  =  pc.get_miu_by_phi
    return prop
end

# 移動係数
function set_function_transfer_coefficient( material_name )
    prop = get_function_material_property()
    prop.lam  =  pc.get_lam
    prop.dw   =  try; pc.get_dw;   catch; pc.get_dw_by_ldml; end
    prop.dp   =  try; pc.get_dp;   catch; pc.get_dp_by_ldmg; end
    prop.ldml =  try; pc.get_ldml; catch; pc.get_ldml_by_dw; end
    prop.ldmg =  try; pc.get_ldmg; catch; pc.get_ldmg_by_dp; end
    prop.ldtg =  try; pc.get_ldtg; catch; pc.get_ldtg_by_dp; end  
    return prop
end

# 値で保持する場合はこちら
function set_value_material_property( cell, material_name )
    prop = get_function_material_property()
    prop.psi  =  pc.get_psi( material_name )
    prop.C    =  pc.get_C( material_name )
    prop.row  =  pc.get_row( material_name )
    prop.crow =  prop.C * prop.row + water_property.Cr * water_property.row * cell.phi
    prop.phi  =  pc.get_phi( cell, material_name )
    prop.dphi =  pc.get_dphi( cell, material_name )
    prop.miu  =  pc.get_miu_by_phi( cell, material_name )
    prop.lam  =  pc.get_lam( cell, material_name )
    prop.dw   =  try; pc.get_dw( cell, material_name );   catch; pc.get_dw_by_ldml( cell, material_name ); end
    prop.dp   =  try; pc.get_dp( cell, material_name );   catch; pc.get_dp_by_ldmg( cell, material_name ); end
    prop.ldml =  try; pc.get_ldml( cell, material_name ); catch; pc.get_ldml_by_dw( cell, material_name ); end
    prop.ldmg =  try; pc.get_ldmg( cell, material_name ); catch; pc.get_ldmg_by_dp( cell, material_name ); end
    prop.ldtg =  try; pc.get_ldtg( cell, material_name ); catch; pc.get_ldtg_by_dp( cell, material_name ); end      
    return prop
end


set_value_material_property (generic function with 1 method)

移動係数の有無は"try/catch"を使って例外処理する。

使用例

In [3]:
mutable struct test_prop
    temp::Float64
    miu::Float64
    rh::Float64
    pv::Float64
    phi::Float64
    test_prop() = new()
end

In [4]:
tp_v1 = test_prop()
tp_v1.temp = 293.15
tp_v1.miu  = -23.0
tp_v1.phi  = pc.get_phi( tp_v1, "benchmark_EN15026" )

0.14599769813374092

In [5]:
tp_v2 = set_value_material_property( tp_v1, "benchmark_EN15026" )

get_function_material_property(0.146, 850.0, 2146.0, 2.435253664272746e6, 0.14599769813374092, 1.6012519825184448e-7, -23.00000000007022, 3.8067636305131067, 1.6151130568635093e-14, 3.0574118219253694e-17, 1.646956969876063e-12, 5.280153258879011e-19, 4.428529887568708e-15)

In [6]:
tp_v2.psi

0.146

In [7]:
tp_v2.C

850.0

In [8]:
tp_v2.row

2146.0

In [9]:
tp_v2.crow

2.435253664272746e6

In [10]:
tp_v2.phi

0.14599769813374092

In [11]:
tp_v2.dphi

1.6012519825184448e-7

In [12]:
tp_v2.miu

-23.00000000007022

In [13]:
tp_v2.lam

3.8067636305131067

In [14]:
tp_v2.dw

1.6151130568635093e-14

In [15]:
tp_v2.dp

3.0574118219253694e-17

In [16]:
tp_v2.ldml

1.646956969876063e-12

In [17]:
tp_v2.ldmg

5.280153258879011e-19

In [18]:
tp_v2.ldtg

4.428529887568708e-15